In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset_path = globals().get("dataset_path")
output_results_path = globals().get("output_results_path")

In [ ]:
full_df = pd.read_csv(dataset_path)

In [ ]:
#full_df = pd.read_csv('/content/drive/MyDrive/Demand Forecasting (Prince Pharma)/Engineered Data/Location Data/California_-_Loc_1_unitsales.csv')

del full_df['ABC_XYZ']
del full_df['Monthwise_SUR']
del full_df['avg_SUR']
del full_df['Quarterly_SUR']
del full_df['ITEMNUMBER']
del full_df['QTY']
del full_df['Revenue']
del full_df['year-month']

In [ ]:
full_df['date_of_sale'] = pd.to_datetime(full_df['date_of_sale'], format="%Y-%m-%d")
full_df['year'] = full_df['date_of_sale'].dt.year
full_df['month'] = full_df['date_of_sale'].dt.month
full_df['quarter'] = full_df['date_of_sale'].dt.quarter

In [ ]:
months_count = full_df.groupby(['item_id'])['date_of_sale'].count().reset_index()
avg_months_count = int(months_count['date_of_sale'].mean())  # Convert mean to integer
avg_months_count

full_df['Monthwise_SUR'] = (full_df['qty_sold'] / (full_df['available_stock'] + full_df['qty_sold'])) * 100
full_df['Monthwise_SUR'] = full_df['Monthwise_SUR'].fillna(0)
months_count = full_df.groupby('item_id')['date_of_sale'].count().reset_index()
months_count = months_count.rename(columns={'date_of_sale': 'months_count'})
avg_utilisation = full_df.groupby('item_id')['Monthwise_SUR'].sum().reset_index()
avg_utilisation = avg_utilisation.merge(months_count, on='item_id', how='left')
avg_utilisation['Avg_Utilisation'] = avg_utilisation['Monthwise_SUR'] / avg_utilisation['months_count']

full_df = full_df.merge(avg_utilisation[['item_id', 'Avg_Utilisation']], on='item_id', how='left')

quarterly_sur = (
    full_df.groupby(['item_id', 'quarter'])['Monthwise_SUR']
    .mean()
    .reset_index()
    .rename(columns={'Monthwise_SUR': 'Quarterly_SUR'})
)
full_df = full_df.merge(quarterly_sur, on=['item_id', 'quarter'], how='left')


full_df = full_df.sort_values(by=['item_id', 'date_of_sale'])

full_df['prev_available_stock'] = full_df.groupby('item_id')['available_stock'].shift(1)
full_df['prev_available_stock'].fillna(method='bfill', inplace=True)  # Use next available value

full_df['prev_qty_sold'] = full_df.groupby('item_id')['qty_sold'].shift(1)
full_df['prev_qty_sold'].fillna(0, inplace=True)  # Default to 0 if missing


# Calculate stock reduction percentage
full_df['stock_reduction'] = np.where(
    full_df['prev_available_stock'] > 0,
    ((full_df['prev_available_stock'] - full_df['available_stock']) / full_df['prev_available_stock']) * 100,
    0  # If previous stock is 0, assume no reduction
)


# Create Stockout Flag (1 if stockout, 0 otherwise)
full_df['stockout_flag'] = ((full_df['available_stock'] == 0) |
                            ((full_df['stock_reduction'] > 50) & (full_df['Monthwise_SUR'] >= 95))).astype(int)
full_df['stockout_flag'].value_counts()

,count
stockout_flag,
0,47007
1,1953


Demand Estimation Trials

In [ ]:
import numpy as np
import pandas as pd

# Sort data to maintain sequential logic
full_df = full_df.sort_values(by=['item_id', 'date_of_sale'])

# Rolling averages for historical demand trends
full_df['rolling_QTY_3M'] = full_df.groupby('item_id')['qty_sold'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)
full_df['rolling_QTY_6M'] = full_df.groupby('item_id')['qty_sold'].rolling(window=6, min_periods=1).mean().reset_index(level=0, drop=True)
full_df['rolling_QTY_12M'] = full_df.groupby('item_id')['qty_sold'].rolling(window=12, min_periods=1).mean().reset_index(level=0, drop=True)

# Default estimated demand = qty_sold
full_df['est_demand'] = full_df['qty_sold']

# ---- Case 1: Adjust demand when stock < qty_sold ----
def adjust_demand(group):
    mask = (group['available_stock'] < group['qty_sold']) & (group['available_stock'] > 0)
    for idx in group[mask].index:
        prev_valid_rows = group.loc[
            (group['date_of_sale'] < group.loc[idx, 'date_of_sale']) &
            (group['qty_sold'] != 0) &
            (group['available_stock'] >= group['qty_sold']) &
            (group['qty_sold'] > group.loc[idx, 'qty_sold'])
        ].sort_values(by='date_of_sale', ascending=False)

        if not prev_valid_rows.empty:
            group.at[idx, 'est_demand'] = prev_valid_rows.iloc[0]['qty_sold']

    return group

full_df = full_df.groupby('item_id', group_keys=False).apply(adjust_demand)

# ---- Case 2: If stock < qty_sold & SUR >= 100%, use previous month's sales ----
full_df['est_demand'] = np.where(
    (full_df['available_stock'] < full_df['qty_sold']) & (full_df['Monthwise_SUR'] >= 100),
    full_df['prev_qty_sold'] + full_df['rolling_QTY_3M'],
    full_df['est_demand']
)

# ---- Case 3: If stock = 0, assume demand based on previous month's qty_sold ----
full_df['est_demand'] = np.where(
    (full_df['available_stock'] == 0),
    full_df['prev_qty_sold'],
    full_df['est_demand']
)

# ---- Case 4: Use 3-month rolling average if stockout persists ----
full_df['est_demand'] = np.where(
    (full_df['available_stock'] == 0) & (full_df['prev_available_stock'] == 0),
    full_df['rolling_QTY_3M'],
    full_df['est_demand']
)

# ---- Case 5: Use 6-month rolling average if stockout persists beyond 3 months ----
full_df['est_demand'] = np.where(
    (full_df['available_stock'] == 0) &
    (full_df['prev_available_stock'] == 0) &
    (full_df['rolling_QTY_3M'].isnull()),
    full_df['rolling_QTY_6M'],
    full_df['est_demand']
)

# ---- Case 6: Use 12-month rolling average if stockout persists beyond 6 months ----
full_df['est_demand'] = np.where(
    (full_df['available_stock'] == 0) &
    (full_df['prev_available_stock'] == 0) &
    (full_df['rolling_QTY_3M'].isnull()) &
    (full_df['rolling_QTY_6M'].isnull()),
    full_df['rolling_QTY_12M'],
    full_df['est_demand']
)

# Ensure all demand estimates are non-negative
full_df['est_demand'] = np.maximum(full_df['est_demand'], 0)


In [ ]:
def add_lag_features(df, target_column='est_demand'):
    """
    Adds lag features for each product separately.
    """
    df = df.sort_values(by=['item_id', 'date_of_sale'])  # Ensure time order per product
    num_months = df['date_of_sale'].nunique()

    for lag in range(1, num_months):
        df[f'lag_{lag}'] = df.groupby('item_id')[target_column].shift(lag)

    df.fillna(0, inplace=True)
    return df
def add_rolling_and_change_features(df, target_column='est_demand'):
    """
    Adds rolling averages, month-over-month (MoM), and year-over-year (YoY) change features per product.
    """
    df = df.sort_values(by=['item_id', 'date_of_sale'])

    # Apply rolling calculations per product
    df['rolling_QTY_3M'] = df.groupby('item_id')[target_column].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)
    df['rolling_QTY_6M'] = df.groupby('item_id')[target_column].rolling(window=6, min_periods=1).mean().reset_index(level=0, drop=True)
    df['rolling_QTY_12M'] = df.groupby('item_id')[target_column].rolling(window=12, min_periods=1).mean().reset_index(level=0, drop=True)

    df['MoM_Change_Absolute'] = df.groupby('item_id')[target_column].diff(1)
    df['YoY_Change_Absolute'] = df.groupby('item_id')[target_column].diff(12)

    df.fillna(0, inplace=True)
    return df

full_df = add_lag_features(full_df)
full_df = add_rolling_and_change_features(full_df)


In [ ]:
!pip install tslearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 7.4 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import StandardScaler
from tslearn.clustering import TimeSeriesKMeans


time_series_rev_df = full_df.groupby(['item_id', 'date_of_sale'])['amount'].sum().reset_index()
pivot_rev_df = time_series_rev_df.pivot(index='item_id', columns='date_of_sale', values='amount').fillna(0)


scaler = StandardScaler()
normalized_rev_data = scaler.fit_transform(pivot_rev_df)
time_series_rev_data = normalized_rev_data.reshape(normalized_rev_data.shape[0], normalized_rev_data.shape[1], 1)


num_abc_clusters = 3  # A, B, C classification
dtw_kmeans_abc = TimeSeriesKMeans(n_clusters=num_abc_clusters, metric="dtw", random_state=42, init='random')
pivot_rev_df['ABC_Cluster_DTW'] = dtw_kmeans_abc.fit_predict(time_series_rev_data)
pivot_rev_df['ABC_Category_DTW'] = pivot_rev_df['ABC_Cluster_DTW'].map({0: 'A', 1: 'B', 2: 'C'})
abc_classification_dtw = pivot_rev_df[['ABC_Category_DTW']].reset_index()

# Convert DATE to datetime format
#full_df['year-month'] = pd.to_datetime(full_df['year-month'])

# Aggregate demand per item over time (daily/weekly demand)
time_series_df = full_df.groupby(['item_id', 'date_of_sale'])['est_demand'].sum().reset_index()

# Pivot to create a time-series format (rows = items, columns = dates)
pivot_df = time_series_df.pivot(index='item_id', columns='date_of_sale', values='est_demand').fillna(0)


# Normalize data (each row represents an item)
scaler = StandardScaler()
normalized_data = scaler.fit_transform(pivot_df)




# Convert data to the correct shape for DTW
time_series_data = normalized_data.reshape(normalized_data.shape[0], normalized_data.shape[1], 1)
num_clusters = 3
# Apply DTW-based K-Means
dtw_kmeans = TimeSeriesKMeans(n_clusters=num_clusters, metric="dtw", random_state=42,init='random')
pivot_df['XYZ_Cluster_DTW'] = dtw_kmeans.fit_predict(time_series_data)

# Map clusters to X, Y, Z
pivot_df['XYZ_Category_DTW'] = pivot_df['XYZ_Cluster_DTW'].map({0: 'X', 1: 'Y',2:'Z'})

# Merge back
xyz_classification_dtw = pivot_df[['XYZ_Category_DTW']].reset_index()

# Step 4: Merge ABC and XYZ classifications
abc_xyz = pd.merge(abc_classification_dtw[['item_id', 'ABC_Category_DTW']], xyz_classification_dtw[['item_id', 'XYZ_Category_DTW']], on='item_id')
# Step 5: Create ABC-XYZ matrix
abc_xyz['ABC_XYZ'] = abc_xyz['ABC_Category_DTW'] + abc_xyz['XYZ_Category_DTW']

# Display the final classification
pd.DataFrame(abc_xyz)
#print(abc_xyz.head())

# Merge ABC-XYZ classification back into the main dataset
full_df = full_df.merge(abc_xyz[['item_id', 'ABC_XYZ']], on='item_id', how='left')
print(pivot_rev_df['ABC_Category_DTW'].value_counts())
print(pivot_df['XYZ_Category_DTW'].value_counts())
print(abc_xyz['ABC_XYZ'].value_counts())

ABC_Category_DTW
B    286
A    222
C    172
Name: count, dtype: int64
XYZ_Category_DTW
X    331
Y    188
Z    161
Name: count, dtype: int64
ABC_XYZ
BX    191
AX    115
CY     89
AZ     62
CZ     58
BY     54
AY     45
BZ     41
CX     25
Name: count, dtype: int64


In [ ]:
from sklearn.preprocessing import LabelEncoder

encoders = {}
reverse_mappings = {}

for col in full_df.columns:
    if full_df[col].dtype == 'object' or full_df[col].apply(lambda x: isinstance(x, str)).any():
        full_df[col] = full_df[col].astype(str)  # Convert entire column to string

        if col == 'ABC_XYZ':  # Apply custom mapping
            abc_xyz_mapping = {
                'AX': 1, 'AY': 2, 'AZ': 3,
                'BX': 4, 'BY': 5, 'BZ': 6,
                'CX': 7, 'CY': 8, 'CZ': 9
            }
            reverse_mapping = {v: k for k, v in abc_xyz_mapping.items()}  # Reverse for decoding
            full_df[col] = full_df[col].map(abc_xyz_mapping)
            reverse_mappings[col] = reverse_mapping

        else:  # Apply Label Encoding for other categorical columns
            encoder = LabelEncoder()
            full_df[col] = encoder.fit_transform(full_df[col]) + 1  # Shift encoding by +1
            encoders[col] = encoder
            reverse_mappings[col] = {idx + 1: label for idx, label in enumerate(encoder.classes_)}  # Reverse mapping

print("\nReverse Mappings:")
for col, mapping in reverse_mappings.items():
    print(f"{col}: {mapping}")



Reverse Mappings:
region: {1: 'California'}
location: {1: 'California - Loc 1'}
item_id: {1: 'Prod_1', 2: 'Prod_10', 3: 'Prod_100', 4: 'Prod_101', 5: 'Prod_102', 6: 'Prod_103', 7: 'Prod_104', 8: 'Prod_105', 9: 'Prod_106', 10: 'Prod_107', 11: 'Prod_108', 12: 'Prod_109', 13: 'Prod_11', 14: 'Prod_110', 15: 'Prod_111', 16: 'Prod_112', 17: 'Prod_113', 18: 'Prod_114', 19: 'Prod_115', 20: 'Prod_116', 21: 'Prod_117', 22: 'Prod_118', 23: 'Prod_119', 24: 'Prod_12', 25: 'Prod_120', 26: 'Prod_121', 27: 'Prod_122', 28: 'Prod_123', 29: 'Prod_124', 30: 'Prod_125', 31: 'Prod_126', 32: 'Prod_127', 33: 'Prod_128', 34: 'Prod_129', 35: 'Prod_13', 36: 'Prod_130', 37: 'Prod_131', 38: 'Prod_132', 39: 'Prod_133', 40: 'Prod_134', 41: 'Prod_135', 42: 'Prod_136', 43: 'Prod_137', 44: 'Prod_138', 45: 'Prod_139', 46: 'Prod_14', 47: 'Prod_140', 48: 'Prod_141', 49: 'Prod_142', 50: 'Prod_143', 51: 'Prod_144', 52: 'Prod_145', 53: 'Prod_146', 54: 'Prod_147', 55: 'Prod_148', 56: 'Prod_149', 57: 'Prod_15', 58: 'Prod_150'

In [ ]:
full_df.replace([np.inf, -np.inf], 0, inplace=True)
df_monthly_train = full_df[full_df['date_of_sale'] < '2024-09-01']
df_monthly_test = full_df[full_df['date_of_sale'] >= '2024-09-01']
df_monthly_train_inverse = df_monthly_train.copy()
df_monthly_train_inverse['item_id'] =  df_monthly_train['item_id'].map(reverse_mappings['item_id'])
df_monthly_train_inverse['location'] =  df_monthly_train['location'].map(reverse_mappings['location'])
df_monthly_train_inverse['region'] =  df_monthly_train['item_id'].map(reverse_mappings['region'])
shop_name = df_monthly_train_inverse['location'].iloc[0]  # Take first row
region_name = df_monthly_train_inverse['region'].iloc[0]

In [ ]:
lost_revenue = ((df_monthly_train['rate']* df_monthly_train['est_demand']) - df_monthly_train['amount']).sum()
lost_revenue

In [ ]:
lost_demand = (df_monthly_train['est_demand']) - df_monthly_train['qty_sold']
lost_demand = lost_demand[lost_demand > 0].sum()
lost_demand

In [ ]:
exclude_cols = [ 'amount', 'date_of_sale','item_id','est_demand','stock_reduction',#'Quarterly_SUR','Monthwise_SUR','Avg_Utilisation',
                'available_stock','stockout_flag','quarter',
                'prev_available_stock','prev_qty_sold','stock_reduction','Quarterly_SUR']
exogenous_features = [col for col in df_monthly_train.select_dtypes(include=['number']).columns if col not in exclude_cols]

In [ ]:
import xgboost as xgb
#from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import warnings
import tensorflow as tf
import lightgbm as lgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import shap
#from prophet import Prophet

from statsmodels.tsa.stattools import adfuller, kpss
warnings.filterwarnings("ignore")
'''
exogenous_features = ['rate', 'holidays', 'discount',
       'loyalty_discount', 'promotions', 'lag_1', 'lag_2', 'lag_3', 'lag_4',
       'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11',
       'lag_12', 'lag_13', 'lag_14', 'lag_15', 'lag_16', 'lag_17', 'lag_18',
       'lag_19', 'lag_20', 'lag_21', 'lag_22', 'lag_23', 'lag_24', 'rolling_QTY_3M',
       'rolling_QTY_6M', 'rolling_QTY_12M', 'MoM_Change_Absolute',
       'YoY_Change_Absolute',  'ABC_XYZ']
'''

print("Exogenous Features:", exogenous_features)

abc_xyz_mapping = {
    1: 'AX',
    2: 'AY',
    3: 'AZ',
    4: 'BX',
    5: 'BY',
    6: 'BZ',
    7: 'CX',
    8: 'CY',
    9: 'CZ'
}


def is_stationary(timeseries):
    """Check if a time series is stationary using ADF and KPSS tests."""
    if timeseries.nunique() == 1:
        return False
    adf_pvalue = adfuller(timeseries)[1]
    try:
        kpss_pvalue = kpss(timeseries, nlags="auto")[1]
    except ValueError:
        kpss_pvalue = 0
    return adf_pvalue < 0.05 and kpss_pvalue > 0.05

def train_lstm(train_x, train_y):
    model = Sequential()
    model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(train_x.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(train_x.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse',metrics='mae')
    model.fit(train_x, train_y, epochs=50, batch_size=16, verbose=0)
    return model

# Function to select top features based on percentage
def select_top_features(train_x, train_y, model, top_percent=10):
    explainer = shap.Explainer(model, train_x)
    shap_values = explainer(train_x, check_additivity=False)

    feature_importance = np.abs(shap_values.values).mean(axis=0)
    feature_importance_df = pd.DataFrame({'feature': train_x.columns, 'importance': feature_importance})

    # Calculate number of features based on percentage
    num_features = max(1, int(len(train_x.columns) * (top_percent / 100)))

    # Select the top N% features
    top_features = feature_importance_df.sort_values(by='importance', ascending=False).head(num_features)['feature'].tolist()

    return top_features

def forecast_product(item_df, abc_xyz_category):
    item = item_df['item_id'].iloc[0]
    item_df = item_df[['date_of_sale', 'est_demand'] + exogenous_features].rename(columns={'date_of_sale': 'ds', 'est_demand': 'y'})
    item_df['ds'] = pd.to_datetime(item_df['ds'])
    item_df1 = item_df[['ds', 'y']]
    # Handle missing regressors
    missing_cols = [col for col in exogenous_features if col not in item_df.columns]
    for col in missing_cols:
        item_df[col] = 0  # Default fill with zero

    # Handle constant regressors
    valid_regressors = item_df[exogenous_features].nunique()[item_df[exogenous_features].nunique() > 1].index.tolist()
    train_x = item_df[valid_regressors]
    train_y = item_df['y']

    model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='mae', random_state=42)
    model.fit(train_x, train_y)

    # Select top 45 features using SHAP
    top_features = select_top_features(train_x, train_y, model, top_percent=10)


    model_type = abc_xyz_mapping.get(abc_xyz_category, 'Default Model')

    if model_type in ["AX","BX","CX"]:
        #model = RandomForestRegressor(random_state=42,criterion='squared_error',max_samples = None, max_depth = None,max_leaf_nodes = None,
          #                            max_features=0.9,bootstrap=True,warm_start=True,oob_score=True)
        #model = lgb.LGBMRegressor(objective='regression', metric='mae', random_state=42,verbose=0)
        model = xgb.XGBRegressor(objective= 'reg:squarederror',eval_metric= 'mae',random_state= 42,learning_rate = 0.1)
        train_x, train_y = item_df[top_features], item_df['y']
        model.fit(train_x, train_y)
        y_pred = model.predict(train_x.iloc[-3:])

    elif model_type in ["AY","BY","CY"]:
        #train_x, train_y = item_df[valid_regressors], item_df['y']
        #lstm_model = train_lstm(train_x, train_y)
        #train_x = np.reshape(train_x.values, (train_x.shape[0], train_x.shape[1], 1))
        #y_pred = lstm_model.predict(train_x[-3:]).flatten()
        #model = xgb.XGBRegressor(objective= 'reg:squarederror',eval_metric= 'mae',random_state= 42,learning_rate = 0.1)
        model = lgb.LGBMRegressor(objective='regression', metric='mae', random_state=42,verbose=-1)
        train_x, train_y = item_df[valid_regressors], item_df['y']
        model.fit(train_x, train_y)
        y_pred = model.predict(train_x.iloc[-3:])
    elif model_type in ["AZ","BZ","CZ"]:
        #model = xgb.XGBRegressor(objective= 'reg:squarederror',eval_metric= 'mae',random_state= 42,
         #                        max_depth = None,reg_alpha = 1,reg_lambda = 1)
        model = RandomForestRegressor(random_state=42,criterion='squared_error',max_samples = None, max_depth = 12,max_leaf_nodes = None,
                                     max_features=0.9,bootstrap=True,warm_start=True,oob_score=True,verbose = 0)
        train_x, train_y = item_df[top_features], item_df['y']
        model.fit(train_x, train_y)
        y_pred = model.predict(train_x.iloc[-3:])
    else:
        model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='mae', random_state=42)
        train_x, train_y = item_df[top_features], item_df['y']
        model.fit(train_x, train_y)
        future_regressors = item_df[valid_regressors].iloc[-3:].reset_index(drop=True)
        y_pred = model.predict(future_regressors)

    y_actual = []  # Store the final 3 values
    historical_years = 3

    for i in range(1, 4):  # Forecast for next 3 months
        past_values = []

        for year in range(1, historical_years + 1):
            past_index = -i - (year * 12)  # Get past value from 1, 2, and 3 years ago

            if abs(past_index) <= len(item_df):  # Ensure index is within bounds
                past_values.append(item_df['y'].iloc[past_index])

        # Compute moving average (or weighted avg if needed)
        if past_values:
            y_actual.append(np.mean(past_values))  # Moving average of past values
        else:
            y_actual.append(0)  # Default to 0 if no past data

    y_actual = np.array(y_actual)


    if np.any(y_actual[:-1] == 0):
        recent_growth_rate = 0
    else:

        weights = np.linspace(1, 2, len(y_actual[12:]))  # Increasing weights (recent data gets higher priority)
        weights = np.linspace(1, 2, len(y_actual[12:]))  # Increasing weights (recent data gets higher priority)

        # Ensure the weights sum to a valid value
        if np.sum(weights) == 0:
            recent_growth_rate = 0  # Default to no trend adjustment
        else:
            recent_growth_rate = np.average(
                (y_actual[12:] / np.clip(y_actual[:-12], 1e-6, None)) - 1,
                weights=weights
            )




    mae = mean_absolute_error(y_actual, y_pred)
    bias = np.mean(y_pred - y_actual)


    if np.isnan(bias) or np.isinf(bias):
        bias = 0

    if np.isnan(recent_growth_rate) or np.isinf(recent_growth_rate):
        recent_growth_rate = 0  # Default to no trend adjustment

    adjusted_forecast = (y_pred - (bias)) * (1 + (recent_growth_rate))
    adjusted_forecast = np.nan_to_num(adjusted_forecast, nan=0, posinf=np.max(y_actual), neginf=np.min(y_actual))
    #adjusted_forecast = y_pred - bias
    adjusted_forecast = np.clip(adjusted_forecast, 0, None)

    return item, y_pred, adjusted_forecast, mae, bias, model_type

results = []
for item, group in df_monthly_train.groupby('item_id'):
    abc_xyz_category = df_monthly_train.loc[df_monthly_train['item_id'] == item, 'ABC_XYZ'].values[0]
    try:
        print(f'Running Forecast for Product: {item} ({abc_xyz_category})')
        results.append(forecast_product(group, abc_xyz_category))
    except Exception as e:
        print(f'Error forecasting for Product {item}: {str(e)}')

# Get the last date from the training data
last_train_month = df_monthly_train['date_of_sale'].max()  # Assuming 'year-month' is a datetime column
last_train_month = pd.to_datetime(last_train_month)  # Ensure it's in datetime format

# Generate forecast month names
forecast_months = [(last_train_month + pd.DateOffset(months=i)).strftime('%b %Y') for i in range(1, 4)]
forecast_results = pd.DataFrame(results, columns=['Encoded_ITEMNUMBER', 'Raw Forecast', 'Adjusted Forecast', 'MAE', 'Bias',
                                                   'model_type'])

forecast_results['item_id'] = forecast_results['Encoded_ITEMNUMBER'].map(reverse_mappings['item_id'])
forecast_results[forecast_months] = forecast_results['Adjusted Forecast'].apply(
    lambda x: pd.Series(x[:3]) if isinstance(x, (list, np.ndarray)) else pd.Series([None, None, None])
)
forecast_results.drop(columns=['Adjusted Forecast', 'Raw Forecast', 'Encoded_ITEMNUMBER'], inplace=True)
forecast_results['location'] = shop_name
forecast_results['region'] = region_name
forecast_results['lost_revenue'] = lost_revenue
forecast_results.head()


Exogenous Features: ['shelf_life', 'region', 'location', 'rate', 'available_stock', 'holidays', 'discount', 'loyalty_discount', 'promotions', 'year', 'month', 'quarter', 'Monthwise_SUR', 'Avg_Utilisation', 'Quarterly_SUR', 'prev_available_stock', 'prev_qty_sold', 'stock_reduction', 'stockout_flag', 'rolling_QTY_3M', 'rolling_QTY_6M', 'rolling_QTY_12M', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12', 'lag_13', 'lag_14', 'lag_15', 'lag_16', 'lag_17', 'lag_18', 'lag_19', 'lag_20', 'lag_21', 'lag_22', 'lag_23', 'lag_24', 'lag_25', 'lag_26', 'lag_27', 'lag_28', 'lag_29', 'lag_30', 'lag_31', 'lag_32', 'lag_33', 'lag_34', 'lag_35', 'lag_36', 'lag_37', 'lag_38', 'lag_39', 'lag_40', 'lag_41', 'lag_42', 'lag_43', 'lag_44', 'lag_45', 'lag_46', 'lag_47', 'lag_48', 'lag_49', 'lag_50', 'lag_51', 'lag_52', 'lag_53', 'lag_54', 'lag_55', 'lag_56', 'lag_57', 'lag_58', 'lag_59', 'lag_60', 'lag_61', 'lag_62', 'lag_63', 'lag_64', 'lag_65', 'lag

,MAE,Bias,model_type,item_id,Sep 2024,Oct 2024,Nov 2024,location,region
0,26.194444,-15.821111,BZ,Prod_1,94.881111,123.071111,109.381111,California - Loc 1,California
1,23.563744,14.017502,BX,Prod_10,79.996468,70.098747,105.904785,California - Loc 1,California
2,80.623271,80.623271,AX,Prod_100,107.337835,107.270681,107.391485,California - Loc 1,California
3,11.801990,8.550266,BX,Prod_101,100.322232,116.438954,81.572148,California - Loc 1,California
4,53.914388,41.430420,BY,Prod_102,112.838466,122.317906,119.843628,California - Loc 1,California


In [ ]:
forecast_results.to_csv(output_results_path, index=False)